# ANALYSIS OF REAL DATA (GSFC)

From real MnKa1, MnKa2 (+other lines) data, it creates a library of optimal filters and reconstruct (and calibrate) data calculating FWHM of lines

**Energy units are (k)eV**

Imports and definitions

PREPROCESSING

1. Clean multiple-pulse records

    1.1. Detect pulses in records file
     
    1.2. Identify multiple pulse records
    
    1.3. Remove multiple-pulse records (to select only single records)

2. Monocrhomatic (Mn Ka1) library creation

    2.1. Plot histogram of maximum values in single-pulse records
    
    2.2. Select records around Ka1 and Ka2 lines (by limiting Pulse Height)
    
    2.3. Create a library with this new file with Kas photons in single records
    
    2.4. Reconstruct Kas data with initial Kas library
    
    2.5. Read data from HR Kas evt file to identify Ka1 and Ka2
    
    2.6. Fit a double Gaussian to select ka1+Ka2 pulses
    
    2.7. Select PH_ID of (non)Ka1 pulses to later exclude them
    
    2.8. Extract Ka1 pulses to build a new more monocrhomatic library
    
    2.9. Build the new libary of Ka1 pulses

PROCESSING

0. Reconstruct data file (singles) with selected record length and library Ka1 and select HR events

1. Read reconstructed events

    1.1. Have a look to events distribution and to variations with baseline and jitter

2. Jitter correction

    2.1 Plot recon PH vs PHASE (distance between trigger and parabola fit=PHI+LAGS) & Fit a polynomial
    
3. Baseline drift correction

    3.1 Plot jiterr_recon PH vs Baseline & Fit polynomial
    
    3.2 Fit gaussians, create new Gain scale and re-calibrate energies
    
4. Fit histogram of baseline-jitter-corrected energies

    4.1. Test different number of bins and see how residuals respond
    
    4.2. Use the range of number of bins where residuals are quite estable and use them to get different FWHM; then take median value
    
5. Recalibrate energies

6. Plot ALL calibrated energies (No correction for non Mn energies)
    - get new gain scale with fitted line centres
    - recalibrate energies
    - fit again to get FWHM more precisely
    
7. Plot FWHM comparison of different reconstruction methods/lengths    
        

## Imports and definitions

In [20]:
from subprocess import check_call, STDOUT
import os
from astropy.io import fits
import numpy.polynomial.polynomial as poly
from numpy.polynomial import Polynomial as P
import tempfile
from datetime import datetime
import shutil, shlex
import numpy as np 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pandas
from RxLines import RxLines
from fit2GaussAndRatio import fit2GaussAndRatio
from getMaximaDensity import getMaximaDensity
from fit2gauss2hist import fit2gauss2hist
from fit3gauss2hist import fit3gauss2hist
from gainScaleFit import gainScaleLinearFit, gainScalePolyFit
from fitVoigt2hist import fitVoigt2hist
from jitterCorr import jitterCorr
from baseCorr import baseCorr
from hdf5_to_fits import hdf5_to_fits
import matplotlib.transforms as transforms
from matplotlib.gridspec import GridSpec
from numpy import random

import ipywidgets as widgets
%matplotlib widget

cwd = os.getcwd()
tmpDir = tempfile.mkdtemp()
os.environ["PFILES"] = tmpDir + ":" + os.environ["PFILES"]
os.environ["HEADASNOQUERY"] = ""
os.environ["HEADASPROMPT"] = "/dev/null/"
xmlfileSX = os.environ["SIXTE"] + "/share/sixte/instruments/athena-xifu/xifu_detector_lpa_75um_AR0.5_pixoffset_mux40_pitch275um_GSFC.xml"
print(xmlfileSX)

/home/ceballos/sw/SIXTE/git/gitInstall/share/sixte/instruments/athena-xifu/xifu_detector_lpa_75um_AR0.5_pixoffset_mux40_pitch275um_GSFC.xml


In [21]:
# data files
channels = (1,3,5,7,9,11,13,15,17,19)
channel = 1
resDir = "channel_" + str(channel)
if not os.path.exists(resDir):
    os.makedirs(resDir)
ratio = 5 # ratio Ka1_gaussProb/Ka2_gaussProb
fileph  = "pulse/pulse_chan" + str(channel) + ".fits" # initial data file with all records and PH_ID column populated
fileph_singles = resDir + "/pulse_chan" + str(channel) + "_singles.fits" # initial data file with all records and PH_ID column populated
filephHR  = resDir + "/pulse_chan" + str(channel) + "_HR.fits" # initial data file with all records and PH_ID column populated and High Res events
fileph_Kas = resDir + "/pulse_chan" + str(channel) + "_Kas.fits" # data file with only those records with Kas lines
noiseph = "noise/noise_chan" + str(channel) + ".fits"
noisefile = "noise/noise_chan" + str(channel) + "_spec.fits"

In [22]:
# Calibration lines CALDB20161122: relative amplitudes of Lorentzians in table = amplitudes of fitted Voigt 
# MnKa
ilabels = ["Ka11","Ka12", "Ka13", "Ka14", "Ka15", "Ka16",  "Ka21", "Ka22"]
energies_eV = np.array([5898.882, 5897.898, 5894.864, 5896.566, 5899.444, 5902.712, 5887.772, 5886.528], dtype=np.float64)
fwhms_eV = np.array([1.7145, 2.0442, 4.4985, 2.6616, 0.97669, 1.5528, 2.3604, 4.2168], dtype=np.float64)
rel_amplitudes = np.array([0.784, 0.263, 0.067, 0.095, 0.071, 0.011, 0.369, 0.1], dtype=np.float64)
MnKas = RxLines(complabel="MnKa", ilabels=ilabels, energies_eV=energies_eV, fwhms_eV=fwhms_eV, rel_amplitudes=rel_amplitudes)
MnKas_cmass = 5894.40 # eV
Ka2_cmass = 5887.37 # eV 
Ka1_cmass = 5898.002 # eV 
#Ka2_cmass = 5887.65 # eV wikipedia nominal
#Ka1_cmass = 5898.75 # eV wikipedia nominal
#Ka2_cmass = 5887.772 # eV most intense
#Ka1_cmass = 5898.882 # eV most intense

# define range to set histogram interval
#min_Ka = 5870 # histogram min energy for MnKa
#max_Ka = 5920 # histogram max energy for MnKa
#min_Kb = 6420 # histogram min energy for MnKb
#max_Kb = 6550 # histogram max energy for MnKb
ilabels = ['Kb1', 'Kb2', 'Kb3', 'Kb4', 'Kb5']
energies_eV = np.array([6490.89, 6486.31, 6477.73, 6490.06, 6488.83], dtype=np.float64)
fwhms_eV = np.array([1.83, 9.4, 13.22, 1.81, 2.81,], dtype=np.float64)
rel_amplitudes = np.array([0.608, 0.109, 0.077, 0.397, 0.176], dtype=np.float64)
MnKb = RxLines(complabel="MnKb", ilabels=ilabels, energies_eV=energies_eV, fwhms_eV=fwhms_eV, rel_amplitudes=rel_amplitudes)
#MnKb_cmass = 6486.38 # eV
MnKb_cmass = 6490.45 # eV "nominal"
#MnKb_cmass = 6490.89 # eV "most intense"
deg = 2 # degree of polynomial for gainscale fit 1=linear; >1: polynomial
lines = (Ka2_cmass, Ka1_cmass, MnKb_cmass)
#lines = (Ka2_cmass, Ka1_cmass)

In [23]:
# Cr-Ka
ilabels = ["Ka11","Ka12", "Ka13", "Ka14", "Ka15", "Ka21", "Ka22"]
energies_eV = np.array([5414.874,5414.099,5412.745,5410.583,5418.304,5405.551,5403.986], dtype=np.float64)
fwhms_eV = np.array([1.457,1.760,3.138,5.149,1.988,2.2224,4.740], dtype=np.float64)
rel_amplitudes = np.array([0.822,0.237,0.085,0.045,0.015,0.386,0.036], dtype=np.float64)
areas = np.array([0.378,0.132,0.084,0.073,0.009,0.271,0.054], dtype=np.float64)                       
CrKas = RxLines(complabel="CrKa", ilabels=ilabels, energies_eV=energies_eV, fwhms_eV=fwhms_eV, rel_amplitudes=rel_amplitudes)
CrKas_cmass = np.sum(energies_eV*areas)/np.sum(areas)
CrKa2_cmass = np.sum(energies_eV[0:5]*areas[0:5])/np.sum(areas[0:5])
CrKa1_cmass = np.sum(energies_eV[5:]*areas[5:])/np.sum(areas[5:])


In [24]:
# select SIRENA parameters for library Kas creation and reconstruction of data files
samprate=195312.5
plen = 4096
oflen = 4096
liblen = 4096 # length of maximum optimal filter for library creation
preBuffer = 0
pBstr = ""
if preBuffer > 0:
    pBstr = "_pB" + str(preBuffer)
method = "OPTFILT"
F0orB0 = "F0"
nS = 5
sU = 3
sD = 4
#KaseV = 5895 # eV reference energy: 8.2%*5.88765(Ka2)+16.2%*5.89875(Ka1) for initial Kas library
libKas = resDir + "/" + "library_Kas_" + str(liblen) + ".fits"

In [25]:
# reconstructed files
evtKas_libKas = resDir + "/" + "evtKas_pulse_chan" + str(channel) + "_libKas_" + "pL" + str(plen) + "_" + method + str(oflen) + ".fits"
evtKas_libKas_HR = resDir + "/" + "evtKas_pulse_chan" + str(channel) + "_libKas_" + "pL" + str(plen) + "_" + method + str(oflen) + "_HR.fits"
evt_libKa1 = resDir + "/" + "evt_pulse_chan" + str(channel) + "_libKa1_" + "pL" + str(plen) + "_" + method + str(oflen) + pBstr + ".fits"
evt_libKa1_HR = resDir + "/" + "evt_pulse_chan" + str(channel) + "_libKa1_" + "pL" + str(plen) + "_" + method + str(oflen) + pBstr + "_HR.fits"


In [26]:
# library Ka1 creation
fileph_Ka1 = resDir + "/" + "pulse_chan" + str(channel) + "_Ka1.fits" # data file with only those single records with Ka1 lines
libKa1 = resDir + "/" + "library_Ka1_"+ str(liblen) + "_chan" + str(channel) +  pBstr + ".fits"

In [27]:
def plotMainXlines(ax, alp_l=0.3, alp_t=0.5):
    """ Plot main X line
    ax: axis of plot
    alp_l: transparency (alpha) of calibration lines markers
    alp_t: transparency (alpha) of calibration lines text
    """
    trans=ax.get_xaxis_transform()
    colors = 3*["green"] + 3*["red"] + 3*["magenta"] + 3*["blue"] + 3*["darkorange"]
    lxx = ["Ka11","Ka21","Kb11"]
    lx = ["Ka1","Ka2","Kb"]
    labels = ["Cr"+ll for ll in lxx] + ["Mn"+ll for ll in lxx] + ["Cu"+ll for ll in lxx] + ["Fe"+ll for ll in lx] + ["Co"+ll for ll in lx]
    xpos   = (5414.874,5405.551,5947.000,  #Cr
              5898.882,5887.772,6490.890,  #Mn
              8047.837,8027.993,8905.532,  #Cu
              6403.840,6390.840,7057.980,  #Fe https://xdb.lbl.gov/Section1/Table_1-2.pdf
              6930.320,6915.300,7649.400)  #Co https://xdb.lbl.gov/Section1/Table_1-2.pdf
    ypos   = np.linspace(0.95,0.05,num=len(colors))
    for i in range(len(colors)):
        if xpos[i] < ax.get_xlim()[0] or xpos[i] > ax.get_xlim()[1]:
            continue
        x=xpos[i]
        ax.axvline(x, color=colors[i], ls=":", alpha=alp_l)
        plt.text(x, ypos[i], labels[i], color=colors[i], transform=trans, alpha=alp_t)

In [28]:
def format_axes(fig):
    for i, ax in enumerate(fig.axes):
        ax.text(0.5, 0.5, "ax%d" % (i+1), va="center", ha="center")
        ax.tick_params(labelbottom=False, labelleft=False)

## PREPROCESSING

## 1. Clean multiple-pulse records

### 1.1) Detect pulses in records file

In [29]:
%%script false --no-raise-error
# build fake library to detect pulses and identify multiple-pulse records
tmpFile = resDir + "/" + "detections0.fits"
tmpFile2 = resDir + "/" + "fakeLib.fits"
comm = ("tesreconstruction Recordfile=" + fileph + " TesEventFile=" + tmpFile + " PulseLength=" + str(liblen) + 
        " LibraryFile=" + tmpFile2 + " samplesUp=" + str(sU) + " nSgms=" + str(nS) + " samplesDown=" + str(sD) + 
        " opmode=0 FilterMethod=" + F0orB0 + " clobber=yes EnergyMethod=" + method + " NoiseFile=" + noisefile +
        " XMLFile=" + xmlfileSX + " monoenergy=" + str(MnKas_cmassass) + " preBuffer=" + str(preBuffer) + " OFLength=" + str(oflen))
try:
    print("Do initial detection")
    print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error Building initial detection-library with command:\n", comm)
    raise
os.remove(tmpFile2)
print("##########################################")
print("Finished creation of fake detection-Library")
print("##########################################")

Do initial detection
tesreconstruction Recordfile=pulse/pulse_chan1.fits TesEventFile=channel_1/detections0.fits PulseLength=4096 LibraryFile=channel_1/fakeLib.fits samplesUp=3 nSgms=5 samplesDown=4 opmode=0 FilterMethod=F0 clobber=yes EnergyMethod=OPTFILT NoiseFile=noise/noise_chan1_spec.fits XMLFile=/home/ceballos/sw/SIXTE/git/gitInstall/share/sixte/instruments/athena-xifu/xifu_detector_lpa_75um_AR0.5_pixoffset_mux40_pitch275um_GSFC.xml monoenergy=5895 preBuffer=0 OFLength=4096
##########################################
Finished creation of fake detection-Library
##########################################


### 1.2) Identify multiple pulse records

In [30]:
#%%script false --no-raise-error

# dump records info
colname = "'SIGNAL, PH_ID, GRADE1, GRADE2'" 
comm = ("fdump wrap=yes infile=" + tmpFile + "+1 columns=" + colname + " rows='-' prhead=no " +
        "showcol=yes showunit=no showrow=no outfile=pulses.txt clobber=yes")
try:
    print("FDUMPing evt file")
    print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error FDUMPing evt file with command:\n", comm)
    shutil.rmtree(tmpDir)
    raise
    


FDUMPing evt file
fdump wrap=yes infile=channel_1/detections0.fits+1 columns='SIGNAL, PH_ID, GRADE1, GRADE2' rows='-' prhead=no showcol=yes showunit=no showrow=no outfile=pulses.txt clobber=yes


In [31]:
#%%script false --no-raise-error

# find single records
dataAll = pandas.read_csv("pulses.txt", skiprows=0, sep="\s+")
display(dataAll)
n_ocurr = dataAll.PH_ID.value_counts() # number of ocurrences of each PH_ID
#print(n_ocurr)
all_PH_ID = np.unique(dataAll.PH_ID.to_list())   # all PH_ID 
single_PH_ID = list()             # single records 
multiple_PH_ID = list()             # multiple records 
for key in sorted(n_ocurr.keys()):
    if n_ocurr[key] == 1:
        #print("Single record for PH_ID=", key)
        single_PH_ID.append(key)
    else:
        multiple_PH_ID.append(key)
#os.remove("pulses.txt")

print("Number of records in",fileph, "=", len(all_PH_ID))
print("Number of Single records in",fileph, "=", len(single_PH_ID))
#print(single_PH_ID)
#print(multiple_PH_ID)
print("Number of Multiple records in",fileph, "=", len(multiple_PH_ID))

,SIGNAL,PH_ID,GRADE1,GRADE2
0,999.0,0,0,4096
1,999.0,1,0,4096
2,999.0,2,0,4096
3,999.0,3,0,4096
4,999.0,4,0,4096
...,...,...,...,...
49025,999.0,47014,0,4096
49026,999.0,47015,0,4096
49027,999.0,47016,0,4096
49028,999.0,47017,0,4096


Number of records in pulse/pulse_chan1.fits = 47007
Number of Single records in pulse/pulse_chan1.fits = 45038
Number of Multiple records in pulse/pulse_chan1.fits = 1969


### 1.3) Remove multiple pulse records (to select only single records)

In [32]:
%%script false --no-raise-error

## select only single records from data file
tmpFile = resDir + "/" + "pp" + str(int(datetime.timestamp(datetime.now()))) + ".fits"
nphs = len(multiple_PH_ID)
# first iteration (requires an initial "'" and input/output files are the original/final)
expr = "'PH_ID != " + str(multiple_PH_ID[0]) + "'"
comm = ("fselect infile=" + fileph + "+1 outfile=" + fileph_singles + " clobber=yes expr=" + expr)
try:
    print("Selecting single-pulse records")
    print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error Selecting single-pulse records with command:\n", comm)
    shutil.rmtree(tmpDir)
    raise
# other non-selections follow: create expressions with 20 selections (ftools breaks if expression is too large)
iph = 1
while iph < nphs:
    expr = "'"
    iiph = 1
    while (iiph < 20 and iph < nphs-1):
        expr = expr + "PH_ID != " + str(multiple_PH_ID[iph]) + " && "
        iiph += 1
        iph  += 1

    expr = expr + "PH_ID != " + str(multiple_PH_ID[iph]) + "'"
    iph += 1
    comm = ("fselect infile=" + fileph_singles + "+1 outfile=" + tmpFile + " clobber=yes expr=" + expr)
    try:
        print("Selecting single-pulse records")
        print("iph=" + str(iph) + "/" + str(nphs))
        print(comm)
        args = shlex.split(comm)
        check_call(args, stderr=STDOUT)
    except RuntimeError:
        print("Error Selecting single-pulse with command:\n", comm)
        shutil.rmtree(tmpDir)
        raise
    shutil.copy(tmpFile, fileph_singles)
    os.remove(tmpFile)

print("Finshed selection of single-pulse records")
# raise SystemExit("Stop after selection of singles")

Selecting single-pulse records
fselect infile=pulse/pulse_chan1.fits+1 outfile=channel_1/pulse_chan1_singles.fits clobber=yes expr='PH_ID != 7'
Selecting single-pulse records
iph=21/1969
fselect infile=channel_1/pulse_chan1_singles.fits+1 outfile=channel_1/pp1592301813.fits clobber=yes expr='PH_ID != 13 && PH_ID != 73 && PH_ID != 81 && PH_ID != 84 && PH_ID != 94 && PH_ID != 109 && PH_ID != 120 && PH_ID != 134 && PH_ID != 146 && PH_ID != 154 && PH_ID != 176 && PH_ID != 178 && PH_ID != 190 && PH_ID != 196 && PH_ID != 245 && PH_ID != 249 && PH_ID != 251 && PH_ID != 262 && PH_ID != 297 && PH_ID != 309'
Selecting single-pulse records
iph=41/1969
fselect infile=channel_1/pulse_chan1_singles.fits+1 outfile=channel_1/pp1592301813.fits clobber=yes expr='PH_ID != 310 && PH_ID != 311 && PH_ID != 317 && PH_ID != 333 && PH_ID != 347 && PH_ID != 395 && PH_ID != 398 && PH_ID != 403 && PH_ID != 405 && PH_ID != 427 && PH_ID != 442 && PH_ID != 466 && PH_ID != 478 && PH_ID != 484 && PH_ID != 487 && PH_ID

## 2. Monochromatic (Ka1) library creation

### 2.1) Plot histogram of maximum values in single-pulse records

In [33]:
f = fits.open(fileph_singles)
ADCdata = f["TESRECORDS"].data['ADC']
baselines = np.mean(ADCdata[:,0:1950], axis=1)
ADCmax = np.amax(ADCdata, axis=1)

fig = plt.figure(figsize=(12,4))
ax1 = fig.add_subplot(1, 2, 1)
bin_heights, bin_borders, _ = ax1.hist(ADCmax, bins=30, alpha=0.4)
#print("MAX_ADC - #")
#for i in range(len(bin_heights)):
#    print(bin_borders[i],bin_heights[i])
ax1.set_xlabel("Maximum value of ADC in record")
ax1.set_ylabel("Number of records")
ax1.set_title("Histogram of max(ADC)")
PHmin = 22500 # ADC units to limit Kas lines
ax1.axvline(PHmin, linestyle="--", color="gray")
PHmax = 25000 # ADC units to limit Kas lines
ax1.axvline(PHmax, linestyle="--", color="gray")

# Identify 'unusual' baselines (fragments of pulses)
ax2 = fig.add_subplot(1, 2, 2)
ax2.hist(baselines, bins=25, alpha=0.4)
ax2.set_title("Histogram of record baselines")
ax2.set_xlabel("Mean value of baseline")
ax2.annotate('partial initial (undetected) pulse', xy=(9500,10000), xytext=(9000, 30000),
            arrowprops=dict(facecolor='black', shrink=0.05),
            )
print("Max baseline=", np.max(baselines))
print("Indices of baselines:", np.where(baselines>8400))
print("Largest baselines:", baselines[baselines>8400])
f.close()
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Max baseline= 10160.496
Indices of baselines: (array([ 1176,  1178,  1179, 13337, 22549, 26148, 30103, 41365]),)
Largest baselines: [ 8400.724  8400.491  8400.977  8752.445  8478.793 10160.496  9010.425
  8465.033]


### 2.2) Select records around Ka1 and Ka2 lines (by limiting the Pulse height)

In [34]:
#%%script false --no-raise-error

comm = ("fselect  infile=" + fileph_singles + "+1 outfile=" + fileph_Kas + " expr='max(ADC)>" + str(PHmin) + " && max(ADC)<" + str(PHmax) + "' clobber=yes")
try:
    print("Selecting Kas by Pulse Height")
    print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error Selecting Kas by Pulse Height with command:\n", comm)
    shutil.rmtree(tmpDir)
    raise

Selecting Kas by Pulse Height
fselect  infile=channel_1/pulse_chan1_singles.fits+1 outfile=channel_1/pulse_chan1_Kas.fits expr='max(ADC)>22500 && max(ADC)<25000' clobber=yes


### 2.3) Create a library with this new file with Kas photons in single records

In [ ]:
%%script false --no-raise-error
# calculate noise spectrum
comm = ("gennoisespec inFile=" + noiseph + " outFile=" + noisefile + " intervalMinSamples=" + str(liblen) +
        " nintervals=1830 pulse_length= clobber=yes samplingRate=" + samprate)
try:
    print("Getting noise spectrum")
    print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error building noise spectrum:\n", comm)
    shutil.rmtree(tmpDir)
    raise

In [35]:
%%script false --no-raise-error

# build library Kas
tmpFile = resDir + "/" + "pp" + str(int(datetime.timestamp(datetime.now()))) + ".fits"
comm = ("tesreconstruction Recordfile=" + fileph_Kas + " TesEventFile=" + tmpFile + " PulseLength=" + str(plen) + 
        " LibraryFile=" + libKas + " samplesUp=" + str(sU) + " nSgms=" + str(nS) + " samplesDown=" + str(sD) + 
        " opmode=0 FilterMethod=" + F0orB0 + " clobber=yes EnergyMethod=" + method + 
        " OFLength=" + str(liblen) + " NoiseFile=" + noisefile +
        " XMLFile=" + xmlfileSX + " monoenergy=" + str(KaseV) + " preBuffer=" + str(preBuffer))
try:
    print("Building initial library (Kas)")
    print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error Building initial library (Kas) with command:\n", comm)
    shutil.rmtree(tmpDir)
    raise
#os.remove(tmpFile)
print("##########################################")
print("Finished creation of Library of Kas events")
print("##########################################")

Building initial library (Kas)
tesreconstruction Recordfile=channel_1/pulse_chan1_Kas.fits TesEventFile=channel_1/pp1592303320.fits PulseLength=4096 LibraryFile=channel_1/library_Kas_4096.fits samplesUp=3 nSgms=5 samplesDown=4 opmode=0 FilterMethod=F0 clobber=yes EnergyMethod=OPTFILT OFLength=4096 NoiseFile=noise/noise_chan1_spec.fits XMLFile=/home/ceballos/sw/SIXTE/git/gitInstall/share/sixte/instruments/athena-xifu/xifu_detector_lpa_75um_AR0.5_pixoffset_mux40_pitch275um_GSFC.xml monoenergy=5895 preBuffer=0
##########################################
Finished creation of Library of Kas events
##########################################


### 2.4) Reconstruct Kas data with initial (Kas) library

In [36]:
#%%script false --no-raise-error
comm = ("tesreconstruction Recordfile=" + fileph_Kas + " TesEventFile=" + evtKas_libKas + " PulseLength=" + str(plen) +
        " LibraryFile=" + libKas + " samplesUp=" + str(sU) + " nSgms=" + str(nS) + " samplesDown=" + str(sD) + " opmode=1 " +
        " clobber=yes EnergyMethod=" + method + " filtEeV=" + str(KaseV) + " OFStrategy=FREE OFLib=yes OFLength=" + str(liblen) + 
        " preBuffer=" + str(preBuffer) + " XMLFile=" + xmlfileSX)

try:
    print("Reconstructing Kas data w/ initial library (Kas)")
    print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error Reconstructing Kas data w/ initial library (Kas) with command:\n", comm)
    shutil.rmtree(tmpDir)
    raise

# Select only HR events
comm = ("fselect  infile=" + evtKas_libKas + " outfile=" + evtKas_libKas_HR + " expr='GRADE1 == " + str(liblen) + 
       " && GRADE2 > 500' clobber=yes")
try:
    print("Selecting HR Kas evts")
    print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error Selecting HR Kas evts with command:\n", comm)
    shutil.rmtree(tmpDir)
    raise

Reconstructing Kas data w/ initial library (Kas)
tesreconstruction Recordfile=channel_1/pulse_chan1_Kas.fits TesEventFile=channel_1/evtKas_pulse_chan1_libKas_pL4096_OPTFILT4096.fits PulseLength=4096 LibraryFile=channel_1/library_Kas_4096.fits samplesUp=3 nSgms=5 samplesDown=4 opmode=1  clobber=yes EnergyMethod=OPTFILT filtEeV=5895 OFStrategy=FREE OFLib=yes OFLength=4096 preBuffer=0 XMLFile=/home/ceballos/sw/SIXTE/git/gitInstall/share/sixte/instruments/athena-xifu/xifu_detector_lpa_75um_AR0.5_pixoffset_mux40_pitch275um_GSFC.xml
Selecting HR Kas evts
fselect  infile=channel_1/evtKas_pulse_chan1_libKas_pL4096_OPTFILT4096.fits outfile=channel_1/evtKas_pulse_chan1_libKas_pL4096_OPTFILT4096_HR.fits expr='GRADE1 == 4096 && GRADE2 > 500' clobber=yes


### 2.5) Read data from HR Kas evt file to identify Ka1 and Ka2

In [37]:
#%%script false --no-raise-error

colname = "'SIGNAL, PH_ID, GRADE1, GRADE2'" 
comm = ("fdump wrap=yes infile=" + evtKas_libKas_HR + "+1 columns=" + colname + " rows='-' prhead=no " +
        "showcol=yes showunit=no showrow=no outfile=pulse.txt clobber=yes")
try:
    print("FDUMPing evt file")
    #print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error FDUMPing evt file with command:\n", comm)
    shutil.rmtree(tmpDir)
    raise
dataKas_HR = pandas.read_csv("pulse.txt", skiprows=0,sep="\s+")
os.remove("pulse.txt")
print("Number of HR Kas pulses in",fileph_Kas, "=", len(dataKas_HR))
print(dataKas_HR)

FDUMPing evt file
Number of HR Kas pulses in channel_1/pulse_chan1_Kas.fits = 18707
         SIGNAL  PH_ID  GRADE1  GRADE2
0      5.851838      0    4096    4096
1      5.839714      1    4096    4096
2      5.849429      2    4096    4096
3      5.846743      3    4096    4096
4      5.846945      4    4096    4096
...         ...    ...     ...     ...
18702  6.334244  47014    4096    4096
18703  5.844837  47015    4096    4096
18704  5.854571  47016    4096    4096
18705  5.849152  47017    4096    4096
18706  5.842519  47018    4096    4096

[18707 rows x 4 columns]


### 2.6) Fit a double Gaussian to select Ka1+Ka2 pulses

Ka1 pulses will be those where the probability Ka1/Ka2 >= 'ratio' 

In [38]:
#%%script false --no-raise-error
plt.close()
nbinsKas = 80

# select data in Kas range (otherwise there is always a smaller pulse comimg from multiple-pulse rows)
# print(min(dataKas_HR.SIGNAL), max(dataKas_HR.SIGNAL))
dataKas_HR = dataKas_HR[(dataKas_HR.SIGNAL>5.8) & (dataKas_HR.SIGNAL<5.89)]
print(min(dataKas_HR.SIGNAL), max(dataKas_HR.SIGNAL))
fig = plt.figure(figsize=(5,3))
ax = fig.add_subplot(1, 1, 1)
ax.hist(dataKas_HR.SIGNAL, bins=50, alpha=0.4, density=True)
ax.set_xlabel("Reconstructed PH (a.u.)")
ax.set_ylabel("# photons")

# Get the range where prob(Ka1)/prob(Ka2)>ratio
(PHminKa1,PHmaxKa1) = fit2GaussAndRatio(data=dataKas_HR.SIGNAL, a1=60, a2=100, mean1=5.845, mean2=5.855, 
                                       sig1=0.005, sig2=0.005, nbins1=nbinsKas, ratio=ratio, xlab="Reconstructed PH (a.u.)",
                                        xlim=(5.81,5.88), ylim=(0,130))
#print("PHminKa1=",PHminKa1, "PHmaxKa1=", PHmaxKa1)

5.800218451016643 5.8899928133473525


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Message (Kas)= The relative error between two consecutive iterates is at most 0.000000
Ka1 PHs in [5.854,5.867] a.u.


### 2.7) Select PH_ID of (non)Ka1 pulses to later exclude them

In [39]:
#%%script false --no-raise-error
print("Number of Kas pulses:", len(dataKas_HR.SIGNAL))
#select PH_IDs for Ka1 (in [PHmin,PHmax] interval)
dataKa1 = dataKas_HR[(dataKas_HR.SIGNAL >= PHminKa1) & (dataKas_HR.SIGNAL <= PHmaxKa1)]
print("Kas pulses which are Ka1 pulses in PHminKa1,PHmaxKa1 interval=", len(dataKa1))

PH_ID_Ka1 = dataKa1.PH_ID.to_list()
#PH_ID_Ka1_single = set(PH_ID_Ka1).intersection(dataKas_single_PH_ID) # good Ka1 pulses

#select PH_IDs to exclude Ka1 in single-pulse records
PH_ID_noKa1 = list(set(dataKas_HR.PH_ID.to_list()).difference(PH_ID_Ka1))
print("Kas pulses which are not non-Ka1 pulses:", len(PH_ID_noKa1))

Number of Kas pulses: 10873
Kas pulses which are Ka1 pulses in PHminKa1,PHmaxKa1 interval= 5021
Kas pulses which are not non-Ka1 pulses: 5852


### 2.8) Extract Ka1 pulses to build a new more monochromatic library

In [40]:
#%%script false --no-raise-error

tmpFile = resDir + "/" + "pp" + str(int(datetime.timestamp(datetime.now()))) + ".fits"
nphs = len(PH_ID_noKa1)
# first iteration (requires an initial "'" and input/output files are the original/final)
expr = "'PH_ID != " + str(PH_ID_noKa1[0]) + "'"
comm = ("fselect infile=" + fileph_Kas + "+1 outfile=" + fileph_Ka1 + " clobber=yes expr=" + expr)
try:
    print("Selecting Ka1 pulses")
    print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error Selecting Ka1 pulses with command:\n", comm)
    shutil.rmtree(tmpDir)
    raise
# other non-selections follow: create expressions with 20 selections (ftools breaks if expression is too large)
iph = 1
while iph < nphs:
    expr = "'"
    iiph = 1
    while (iiph < 20 and iph < nphs-1):
        expr = expr + "PH_ID != " + str(PH_ID_noKa1[iph]) + " && "
        iiph += 1
        iph  += 1

    expr = expr + "PH_ID != " + str(PH_ID_noKa1[iph]) + "'"
    iph += 1
    comm = ("fselect infile=" + fileph_Ka1 + "+1 outfile=" + tmpFile + " clobber=yes expr=" + expr)
    try:
        print("Selecting Ka1 pulses")
        print("iph=" + str(iph) + "/" + str(nphs))
        print(comm)
        args = shlex.split(comm)
        check_call(args, stderr=STDOUT)
    except RuntimeError:
        print("Error Selecting Ka1 pulses with command:\n", comm)
        shutil.rmtree(tmpDir)
        raise
    shutil.copy(tmpFile, fileph_Ka1)
    os.remove(tmpFile)

print("Finshed selection of Ka1 events")
# raise SystemExit("Stop after selection of Ka1")

Selecting Ka1 pulses
fselect infile=channel_1/pulse_chan1_Kas.fits+1 outfile=channel_1/pulse_chan1_Ka1.fits clobber=yes expr='PH_ID != 0'
Selecting Ka1 pulses
iph=21/5852
fselect infile=channel_1/pulse_chan1_Ka1.fits+1 outfile=channel_1/pp1592303551.fits clobber=yes expr='PH_ID != 1 && PH_ID != 2 && PH_ID != 3 && PH_ID != 4 && PH_ID != 5 && PH_ID != 6 && PH_ID != 8 && PH_ID != 9 && PH_ID != 32770 && PH_ID != 11 && PH_ID != 12 && PH_ID != 32775 && PH_ID != 15 && PH_ID != 16 && PH_ID != 17 && PH_ID != 18 && PH_ID != 32778 && PH_ID != 20 && PH_ID != 21 && PH_ID != 32781'
Selecting Ka1 pulses
iph=41/5852
fselect infile=channel_1/pulse_chan1_Ka1.fits+1 outfile=channel_1/pp1592303551.fits clobber=yes expr='PH_ID != 23 && PH_ID != 32785 && PH_ID != 26 && PH_ID != 27 && PH_ID != 28 && PH_ID != 29 && PH_ID != 30 && PH_ID != 32790 && PH_ID != 32 && PH_ID != 33 && PH_ID != 34 && PH_ID != 35 && PH_ID != 36 && PH_ID != 37 && PH_ID != 39 && PH_ID != 40 && PH_ID != 42 && PH_ID != 43 && PH_ID != 45 &&

### 2.9) Build the new library of Ka1 pulses

In [41]:
#%%script false --no-raise-error
tmpFile = resDir + "/" + "pp" + str(int(datetime.timestamp(datetime.now()))) + ".fits"
comm = ("tesreconstruction Recordfile=" + fileph_Ka1 + " TesEventFile=" + tmpFile + " PulseLength=" + str(liblen) +
        " LibraryFile=" + libKa1 + " samplesUp=" + str(sU) + " nSgms=" + str(nS) + " samplesDown=" + str(sD) +
        " opmode=0" + " FilterMethod=" + F0orB0 + " clobber=yes" + " EnergyMethod=" + method +  " NoiseFile=" + noisefile + 
        " XMLFile=" + xmlfileSX + " monoenergy=" + str(Ka1_cmass) + " preBuffer=" + str(preBuffer))
try:
    print("Building new library (Ka1)")
    print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error Building new library (Ka1) with command:\n", comm)
    os.remove(tmpFile)
    raise
os.remove(tmpFile)

Building new library (Ka1)
tesreconstruction Recordfile=channel_1/pulse_chan1_Ka1.fits TesEventFile=channel_1/pp1592304347.fits PulseLength=4096 LibraryFile=channel_1/library_Ka1_4096_chan1.fits samplesUp=3 nSgms=5 samplesDown=4 opmode=0 FilterMethod=F0 clobber=yes EnergyMethod=OPTFILT NoiseFile=noise/noise_chan1_spec.fits XMLFile=/home/ceballos/sw/SIXTE/git/gitInstall/share/sixte/instruments/athena-xifu/xifu_detector_lpa_75um_AR0.5_pixoffset_mux40_pitch275um_GSFC.xml monoenergy=5898.002 preBuffer=0


# PROCESSING

## 0. Reconstruct data file (singles) with selected record length and select HR events
(try different record lengths to get variation of FWHM)

In [42]:
#%%script false --no-raise-error
# reconstruct pulses according to input pulseLength and OFlength
comm = ("tesreconstruction Recordfile=" + fileph_singles + " TesEventFile=" + evt_libKa1 + " PulseLength=" + str(plen) +
        " LibraryFile=" + libKa1 + " samplesUp=" + str(sU) + " nSgms=" + str(nS) + " samplesDown=" + str(sD) +
        " opmode=1 clobber=yes EnergyMethod=" + method + " XMLFile=" + xmlfileSX + " LbT=0.01" + 
        " filtEeV=" + str(Ka1_cmass) + " OFStrategy=FIXED OFLength=" + str(oflen) + " preBuffer=" + str(preBuffer))
try:
    print("Reconstructing real data w/ library (Ka1)")
    print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error Reconstructing real data w/ library (Ka1) with command:\n", comm)
    shutil.rmtree(tmpDir)
    raise   
print("########################")
print("Reconstruction finished")
print("########################")

Reconstructing real data w/ library (Ka1)
tesreconstruction Recordfile=channel_1/pulse_chan1_singles.fits TesEventFile=channel_1/evt_pulse_chan1_libKa1_pL4096_OPTFILT4096.fits PulseLength=4096 LibraryFile=channel_1/library_Ka1_4096_chan1.fits samplesUp=3 nSgms=5 samplesDown=4 opmode=1 clobber=yes EnergyMethod=OPTFILT XMLFile=/home/ceballos/sw/SIXTE/git/gitInstall/share/sixte/instruments/athena-xifu/xifu_detector_lpa_75um_AR0.5_pixoffset_mux40_pitch275um_GSFC.xml LbT=0.01 filtEeV=5898.002 OFStrategy=FIXED OFLength=4096 preBuffer=0
########################
Reconstruction finished
########################


In [43]:
# select only HR events
comm = ("fselect  infile=" + evt_libKa1 + " outfile=" + evt_libKa1_HR + " expr='GRADE1 >= " + str(min(plen,oflen)) + 
       " && GRADE2 > 500' clobber=yes")
try:
    print("Selecting HR evts")
    print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error Selecting HR evts with command:\n", comm)
    shutil.rmtree(tmpDir)
    raise

Selecting HR evts
fselect  infile=channel_1/evt_pulse_chan1_libKa1_pL4096_OPTFILT4096.fits outfile=channel_1/evt_pulse_chan1_libKa1_pL4096_OPTFILT4096_HR.fits expr='GRADE1 >= 4096 && GRADE2 > 500' clobber=yes


## 1. Read reconstructed events

In [44]:
colname = "'SIGNAL, PH_ID, GRADE1, GRADE2, PHI, LAGS, BSLN'" 
comm = ("fdump wrap=yes infile=" + evt_libKa1_HR + "+1 columns=" + colname + " rows='-' prhead=no " +
        "showcol=yes showunit=no showrow=no outfile=pulse.txt clobber=yes pagewidth=256")
try:
    print("FDUMPing evt file")
    print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error FDUMPing evt file with command:\n", comm)
    shutil.rmtree(tmpDir)
    raise
data_HR = pandas.read_csv("pulse.txt", skiprows=0,sep="\s+")
print("\nNumber of initial (all energies) HR pulses:", len(data_HR)) 
#os.remove("pulse.txt")
data_HR

# exclude events with PHI=0. It'll mean that there is a pulse (undetected) over the rise of another one
data_HR = data_HR[(data_HR.PHI>0.) | (data_HR.PHI<0)]
# exclude high baseline events: they have a truncated pulse at the beginnig of the record
data_HR = data_HR[(data_HR.BSLN<8325)]

print("\nNumber of non-excluded events (all energies) HR pulses:", len(data_HR)) 
print("MinSIGNAL, MaxSIGNAL=", min(data_HR.SIGNAL), max(data_HR.SIGNAL))
print("MinBSLN, MaxBSLN=", min(data_HR.BSLN), max(data_HR.BSLN))
data_HR


FDUMPing evt file
fdump wrap=yes infile=channel_1/evt_pulse_chan1_libKa1_pL4096_OPTFILT4096_HR.fits+1 columns='SIGNAL, PH_ID, GRADE1, GRADE2, PHI, LAGS, BSLN' rows='-' prhead=no showcol=yes showunit=no showrow=no outfile=pulse.txt clobber=yes pagewidth=256

Number of initial (all energies) HR pulses: 45038

Number of non-excluded events (all energies) HR pulses: 43808
MinSIGNAL, MaxSIGNAL= 0.2555883435612861 11.38430180953629
MinBSLN, MaxBSLN= 8263.99334357399 8324.989759344598


,SIGNAL,PH_ID,GRADE1,GRADE2,PHI,LAGS,BSLN
955,5.834600,1022,4096,4096,0.063009,0,8324.989759
970,5.828243,1037,4096,4096,0.180935,0,8324.566308
977,5.823795,1044,4096,4096,-0.148047,0,8324.172043
978,5.830708,1045,4096,4096,0.119082,0,8324.310804
981,5.832259,1048,4096,4096,-0.179209,0,8322.301075
...,...,...,...,...,...,...,...
45033,6.314550,47014,4096,4096,0.178702,0,8286.372760
45034,5.826560,47015,4096,4096,0.455115,0,8286.811572
45035,5.836268,47016,4096,4096,-0.071194,0,8285.674859
45036,5.830864,47017,4096,4096,-0.210358,0,8308.199693


### 1.1 Have a look to events distribution and to variations with baseline and jitter

In [ ]:
# main lines in reconstructed PH units
PHlines = dict()
PHlines = {"Sc-Ka": (4.225,4.300),
           "Ti-Ka": (4.615,4.667),
           "V-Ka":  (5.000,5.070),
           "Cr-Ka": (5.420,5.440),
           "Mn-Ka": (5.800, 5.850),
           "Fe-Ka": (6.200,6.270),
           "Co-Ka": (6.590,6.710),
           "Ni-Ka": (7.000,7.150),
           "Cu-Ka": (7.400,7.540),
           "Zn-Ka": (7.846,8.000),
           "Ge-Ka": (8.631,8.825),
           "Br-Ka": (9.895,10.109)
          }


In [ ]:
#%%script false --no-raise-error

# Events PH distribution
plt.close()
fig = plt.figure(constrained_layout=True, figsize=(10,8))
gs = GridSpec(2, 2, figure=fig)

# Have a look to histogram of ALL reconstructed data
#fig = plt.figure(figsize=(10,8))
#ax1 = fig.add_subplot(2, 2, 1)
ax1 = fig.add_subplot(gs[0, :])
npulses = len(data_HR.SIGNAL)
ax1.set_xlabel("Reconstructed PH (ma.u.)")
ax1.set_ylabel("#events")
ax1.set_title(("Distribution of ALL events (" + str(npulses) + ")"))
ax1.hist(1e3*data_HR.SIGNAL, bins=2000, density=False, label="Histogram", alpha=0.5, log=True)
ax1.set_xlim(2000,11000)
for line in PHlines:
    PHmin,PHmax = PHlines[line]
    if line == "Mn-Ka":
        ax1.text(1e3*PHmin, 4e3, line)
    else:
        ax1.text(1e3*PHmin, 3e3*random.uniform(0.1,0.7), line)
#ax1.set_xlim(5000,14000) # 0-pad 2048
#plotMainXlines(ax1)

# Have a look to histogram of Kas
#ax3 = fig.add_subplot(2, 2, 3)
ax2 = fig.add_subplot(gs[1, 0])
#(minKas,maxKas) = (5.66, 5.75)
(minKas,maxKas) = (5.77, 5.87)
#(minKas,maxKas) = (8.600, 8.900) # 0-pad2048
#(minKas,maxKas) = (9.750, 9.900) # 0-pad 1024
npulses = len(data_HR[(data_HR.SIGNAL>minKas) & (data_HR.SIGNAL<maxKas)].SIGNAL)
ax2.hist(1e3*data_HR[(data_HR.SIGNAL>minKas) & (data_HR.SIGNAL<maxKas)].SIGNAL, bins=100, density=False, label="Histogram", alpha=0.5, color="tab:red")
ax2.set_xlabel("Reconstructed PH (a.u.)")
ax2.set_ylabel("# events")
ax2.set_title(("Distribution of Mn Kas events (" + str(npulses) + ")"))
ax2.set_xlim(5770,5920)
#ax3.set_xlim(8650,8900) # 0-pad 2048
#ax3.set_xlim(9750,9900) # 0-pad 1024
#plotMainXlines(ax2)

# Have a look to histogram of Kb
#ax4= fig.add_subplot(2, 2, 4)
ax3 = fig.add_subplot(gs[1, 1])
(minKb,maxKb) = (6.28, 6.35)
#(minKb,maxKb) = (9.210, 9.310) # 0-pad 2048
#(minKb,maxKb) = (10.275, 10.350) # 0-pad
npulses = len(data_HR[(data_HR.SIGNAL>minKb) & (data_HR.SIGNAL<maxKb)].SIGNAL)
ax3.hist(1e3*data_HR[(data_HR.SIGNAL>minKb) & (data_HR.SIGNAL<maxKb)].SIGNAL, bins=100, density=False, label="Histogram", alpha=0.5, color="tab:red")
ax3.set_xlabel("Reconstructed PH (a.u.)")
ax3.set_ylabel("# events")
ax3.set_title(("Distribution of Mn Kb events (" + str(npulses) + ")"))
ax3.set_xlim(6280,6350)
#ax4.set_xlim(9210,9310) # 0-pad 2048
#ax4.set_xlim(10275,10350) # 0-pad 1024
#plotMainXlines(ax3)



In [ ]:
fig = plt.figure(figsize=(10,10))
i = 0
for line in PHlines:
    i+=1
    color="C"+str(i)
    print("Plotting line=", PHlines[line])
    PHmin,PHmax = PHlines[line]
    npulses = len(data_HR[(data_HR.SIGNAL>PHmin) & (data_HR.SIGNAL<PHmax)].SIGNAL)
    ax = fig.add_subplot(4, 3, i)
    dataLine = data_HR[(data_HR.SIGNAL>PHmin) & (data_HR.SIGNAL<PHmax)].SIGNAL
    ax.hist(1e3*dataLine, bins=100, density=False, alpha=0.5, color=color, label=line)
    ax.set_xlabel("Reconstructed PH (a.u.)")
    ax.set_ylabel("# events")
    ax.set_xlim(1e3*PHmin,1e3*PHmax)
    ax.legend()

fig.tight_layout()


In [12]:
# 4096
PHminKas = 5.80 #5.66
PHmaxKas = 5.86 #5.73
PHminKb = 6.28 #6.13
PHmaxKb = 6.34 #6.20

    # 1024
#PHminKas = 5.80 
#PHmaxKas = 5.87 
#PHminKb = 6.25 
#PHmaxKb = 6.31 

# 2048
#PHminKas = 5.80 
#PHmaxKas = 5.86 
#PHminKb = 6.27
#PHmaxKb = 6.33 

# 0-padding 2048
#PHminKas = 8.7 
#PHmaxKas = 8.85 
#PHminKb = 9.21 
#PHmaxKb = 9.28 

# 0-padding 1024
#PHminKas = 9.78
#PHmaxKas = 9.86 
#PHminKb = 10.28
#PHmaxKb = 10.33 

In [ ]:
# plot jitters
plt.close()
phase_HR = data_HR.PHI + data_HR.LAGS
base_HR = data_HR.BSLN

fig = plt.figure(figsize=(11,10))
i = 0
for line in PHlines:
    i+=1
    print("Plotting line=", PHlines[line])
    PHmin,PHmax = PHlines[line]
    dataLine = data_HR[(data_HR.SIGNAL>PHmin) & (data_HR.SIGNAL<PHmax)]
    dataLine_norm = dataLine.SIGNAL/(np.median(dataLine.SIGNAL))
    phase_HR = dataLine.PHI + dataLine.LAGS
    phase_HR = dataLine.PHI 
    ax1 = fig.add_subplot(4, 3, i)
    ax1.set_xlabel("Phase (samples)")
    ax1.set_ylabel("Reconstructed PH (a.u.)")
    title = "Distribution of " + line + " events"
    ax1.set_title(title)
    ax1.scatter(phase_HR, dataLine_norm, alpha=0.5)
    #ax1.scatter(phase_HR, dataLine.LAGS, alpha=0.5)

fig.tight_layout()

In [ ]:
# Jitter and baseline distribution
plt.close()
phase_HR = data_HR.PHI + data_HR.LAGS
base_HR = data_HR.BSLN

fig = plt.figure(figsize=(10,4))
ax1 = fig.add_subplot(1, 2, 1)
ax1.set_xlabel("Phase (samples)")
ax1.set_ylabel("Reconstructed PH (a.u.)")
ax1.set_title("Distribution of ALL events")
ax1.scatter(phase_HR, data_HR.SIGNAL, alpha=0.5)

ax2 = fig.add_subplot(1, 2, 2)
ax2.set_xlabel("Baseline (ADC a.u.)")
ax2.set_ylabel("Reconstructed PH (a.u.)")
ax2.set_title("Distribution of ALL events")
ax2.scatter(base_HR, data_HR.SIGNAL, alpha=0.1)
fig.tight_layout()

## 2. Jitter correction

### 2.1 Plot Pulse Height vs PHASE (distance between trigger and parabola fit = PHI + LAGS) & fit polynomial

In [45]:
plt.close()
dataKas_HR = data_HR[(data_HR.SIGNAL>PHminKas) & (data_HR.SIGNAL<PHmaxKas)]
dataKb_HR = data_HR[(data_HR.SIGNAL>PHminKb) & (data_HR.SIGNAL<PHmaxKb)]

print("Number of pulses in dataKas_HR: ", len(dataKas_HR))
phaseKas_HR = dataKas_HR.PHI + dataKas_HR.LAGS
dataKas_HR_jitter = jitterCorr(reconPH=dataKas_HR.SIGNAL, phase=phaseKas_HR, xsize=10, ysize=4, alpha=0.2)

print("\nNumber of pulses in dataKb_HR: ", len(dataKb_HR))
phaseKb_HR = dataKb_HR.PHI + dataKb_HR.LAGS
dataKb_HR_jitter = jitterCorr(reconPH=dataKb_HR.SIGNAL, phase=phaseKb_HR, xsize=10, ysize=4, alpha=0.2)

Number of pulses in dataKas_HR:  9839


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fit PH=5.834+ (-0.000)*x**1+ (0.004)*x**2
Fit PH corrected=5.834+ (-0.000)*x**1+ (0.000)*x**2

Number of pulses in dataKb_HR:  1409


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fit PH=6.311+ (-0.002)*x**1+ (0.005)*x**2
Fit PH corrected=6.312+ (0.000)*x**1+ (-0.000)*x**2


## 3. Baseline drift correction

### 3.1 Plot jiterr_recon PH vs Baseline & Fit polynomial


In [46]:
plt.close()
print("Mn Kas")
baseKas_HR = (dataKas_HR.BSLN)
dataKas_HR_jitter_bsln = baseCorr(reconPH=dataKas_HR_jitter, base=baseKas_HR, xsize=10, ysize=4, alpha=0.2)

print("\nMn Kb")
baseKb_HR = (dataKb_HR.BSLN)
dataKb_HR_jitter_bsln = baseCorr(reconPH=dataKb_HR_jitter, base=baseKb_HR, xsize=10, ysize=4, alpha=0.2)


Mn Kas


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fit PH=-272.141+ (0.067)*x**1+ (-0.000)*x**2
Fit PH corrected=5.834+ (0.000)*x**1+ (-0.000)*x**2

Mn Kb


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fit PH=-165.930+ (0.042)*x**1+ (-0.000)*x**2
Fit PH corrected=6.312+ (-0.000)*x**1+ (0.000)*x**2


### 3.2 Fit gaussians, create Gain scale and calibrate energies

In [47]:
plt.close()
nbinsKas = 100
nbinsKb = 50
#print(min(dataKas_HR_jitter_bsln), max(dataKas_HR_jitter_bsln))
# use kernel density function to get an initial (automatic) estimation of the centres of the lines
#centresA = getMaximaDensity(data=1e3*dataKas_HR_jitter_bsln, nmaxima=2, intervalmin=5600, intervalmax=5780)
centresA=[5830,5838] #[5700,5710] 
centresB=6314

# 0-padding 2048
#centresA=[8772,8781] #[5700,5710] 
#centresB=9265

# 0-padding 1024
#centresA=[9827,9836] #[5700,5710] 
#centresB=10315

print("Estimated initial values for MnKa2 and MnKa1=",centresA)

(mean1bsln, mean2bsln, mean3bsln) = fit3gauss2hist(data1=1e3*dataKas_HR_jitter_bsln, data2=1e3*dataKb_HR_jitter_bsln,
                                        a1=0.06, a2=0.12, a3=0.1, sig1=5, sig2=5, sig3=5, 
                                        mean1=centresA[0], mean2=centresA[1], mean3=centresB,
                                        nbins1=nbinsKas, nbins2=nbinsKb, 
                                        xlab="reconstructed PH (a.u.)", plot=True, xsize=12,ysize=4)


Estimated initial values for MnKa2 and MnKa1= [5830, 5838]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
# create gain scale Ka2, Ka1, Kb
plt.close()
recon_lines = (mean1bsln,mean2bsln, mean3bsln)
coefs = gainScalePolyFit(xData=recon_lines, yData=lines, deg=2, ylab="MnK Lines energies (eV)", xsize=10, ysize=4)
print("gain scale coefs=", coefs)

RMSE: 3.5224578334304324e-12
R-squared: 1.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

gain scale coefs= [-1.40016665e+03  1.25679123e+00 -1.14197005e-06]


In [49]:
min_Ka=5870
max_Ka=5910
min_Kb=6470
max_Kb=6505
# recalibrate energies
enerKas_HR_jitter_bsln = np.zeros(len(dataKas_HR_jitter_bsln))
enerKb_HR_jitter_bsln = np.zeros(len(dataKb_HR_jitter_bsln))

for i in range(len(coefs)):
    enerKas_HR_jitter_bsln += coefs[i] * (1e3*dataKas_HR_jitter_bsln)**(i)
enerKas_HR_jitter_bsln = enerKas_HR_jitter_bsln[(enerKas_HR_jitter_bsln > min_Ka) & (enerKas_HR_jitter_bsln < max_Ka)]

for i in range(len(coefs)):
    enerKb_HR_jitter_bsln += coefs[i] * (1e3*dataKb_HR_jitter_bsln)**(i)
enerKb_HR_jitter_bsln = enerKb_HR_jitter_bsln[(enerKb_HR_jitter_bsln>min_Kb) & (enerKb_HR_jitter_bsln<max_Kb)]
#print(enerKb_HR_jitter_bsln)

## 4 Fit histogram of baseline-jitter-corrected energies

#### 4.1 Test different number of bins and see how the residuals respond

In [50]:
#%%script false --no-raise-error
# Plot residuals curve to select best number of bins for Kas and Kb
plt.clf()
fig = plt.figure(figsize=(10, 4))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

fitVoigt2hist(data=enerKas_HR_jitter_bsln, lines=MnKas, nbins=np.arange(30,200,20), ax0=ax1)
#fitVoigt2hist(data=enerKas_HR_jitter_bsln, lines=MnKas, nbins=50, ax0=ax1)
ax1.set_title("MnKas")

fitVoigt2hist(data=enerKb_HR_jitter_bsln, lines=MnKb, nbins=np.arange(20,100,10), ax0=ax2)
ax2.set_title("MnKb")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'MnKb')

#### 4.2 Use the range of number of bins where residuals are quite estable and use them to get different FWHM; then take median value

In [51]:
#%%script false --no-raise-error
plt.close()
# Calculate mean 'fit' for range of acceptable number of bins in Kas range
#                                                               ===========
(ibmin, ibmax) = (50, 200)
fwhm_G = list()
err_fwhm_G = list()
nbins = list()
for nbinsKas in np.arange(ibmin, ibmax, 10):
    fw, efw, vv_fit = fitVoigt2hist(data=enerKas_HR_jitter_bsln, lines=MnKas, nbins=int(nbinsKas), ax0=ax1)    
    plt.clf()
    if fw < 0 or efw == 0:
        continue
    fwhm_G.append(fw)
    err_fwhm_G.append(efw)
    nbins.append(nbinsKas)
print("FWHM_Gs=", np.array2string(np.asarray(fwhm_G), formatter={'float_kind':lambda x: "%.3f" % x}))
median_fwhm_G = np.median(fwhm_G)
median_err_fwhm_G = np.sum(abs(np.asarray(fwhm_G)-median_fwhm_G))/len(fwhm_G)
median_index = min(range(len(fwhm_G)), key=lambda i: abs(fwhm_G[i]-median_fwhm_G))
nbinsKas = nbins[median_index]
print("median_index=", median_index)
print("FWHM_G[median_index]=",fwhm_G[median_index])
print("ERR_FWHM_G[median_index]=",err_fwhm_G[median_index])
print("nbinsKas[median_index]=", nbins[median_index])

print("Using ", len(fwhm_G), "different number of bins")
txt_Kas = '{:0.2f}'.format(median_fwhm_G) + "+/-" + '{:0.2f}'.format(max(median_err_fwhm_G, err_fwhm_G[median_index])) + "eV"
print("Median FWHM_G(Kas)=", txt_Kas)


FWHM_Gs= [3.519 3.538 3.528 3.539 3.503 3.517 3.506 3.514 3.501 3.521 3.505 3.509
 3.511]
median_index= 7
FWHM_G[median_index]= 3.5141457177211057
ERR_FWHM_G[median_index]= 0.08828838397025407
nbinsKas[median_index]= 130
Using  13 different number of bins
Median FWHM_G(Kas)= 3.51+/-0.09eV


In [ ]:
#%%script false --no-raise-error

# Calculate mean 'fit' for range of acceptable number of bins in Kb range
#                                                               ===========
(ibmin, ibmax) = (30, 100)
fwhm_G = list()
err_fwhm_G = list()
nbins = list()
for nbinsKb in np.arange(ibmin, ibmax, 10):
    fw, efw, vv_fit = fitVoigt2hist(data=enerKb_HR_jitter_bsln, lines=MnKb, nbins=int(nbinsKb), ax0=ax2)    
    plt.clf()
    if efw == 0 or fw < 0:
        continue
    fwhm_G.append(fw)
    err_fwhm_G.append(efw)
    nbins.append(nbinsKb)
print("FWHM_Gs=", np.array2string(np.asarray(fwhm_G), formatter={'float_kind':lambda x: "%.3f" % x}))
median_fwhm_G = np.median(fwhm_G)
median_err_fwhm_G = np.sum(abs(np.asarray(fwhm_G)-median_fwhm_G))/len(fwhm_G)
median_index = min(range(len(fwhm_G)), key=lambda i: abs(fwhm_G[i]-median_fwhm_G))
nbinsKb = nbins[median_index]
print("median_index=", median_index)
print("FWHM_G[median_index]=",fwhm_G[median_index])
print("ERR_FWHM_G[median_index]=",err_fwhm_G[median_index])
print("nbinsKb[median_index]=", nbins[median_index])

txt_Kb = '{:0.2f}'.format(median_fwhm_G) + "+/-" + '{:0.2f}'.format(max(median_err_fwhm_G, err_fwhm_G[median_index])) + "eV"
print("Median FWHM_G(Kb)=", txt_Kb)

In [ ]:
#%%script false --no-raise-error
plt.close()
# Plot representative (median) fitting 
fig = plt.figure(figsize=(12, 6))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

print("Using nbinsKas=", nbinsKas, "from median estimation")
print("Using nbinsKb=", nbinsKb, "from median estimation")

nlines_Kas = MnKas.getNumber() 
nlines_Kb = MnKb.getNumber()
nlines = nlines_Kas + nlines_Kb 
lines_centres_vv = np.zeros(nlines)

fwhm_G, err_fwhm_G, vvmod = fitVoigt2hist(data=enerKas_HR_jitter_bsln, lines=MnKas, nbins=int(nbinsKas), ax0=ax1)
print(fwhm_G, err_fwhm_G)
for i in range(nlines_Kas):
     lines_centres_vv[i] = vvmod.param_sets[i*4][0]

fwhm_G, err_fwhm_G, vvmod = fitVoigt2hist(data=enerKb_HR_jitter_bsln, lines=MnKb, nbins=int(nbinsKb), ax0=ax2)
print(fwhm_G, err_fwhm_G)
for i in range(nlines_Kb):
    j = i + nlines_Kas
    lines_centres_vv[j] = vvmod.param_sets[i*4][0]

x1 = min(enerKas_HR_jitter_bsln)
y1 = 20
ax1.text(x1,y1, "FWHM=" + txt_Kas + "\n Total counts=" + str(len(enerKas_HR_jitter_bsln)))
ax1.axvline(MnKas.energies_eV[0], ls="--", color="gray")

x2 = min(enerKb_HR_jitter_bsln)
y2 = 80
ax2.text(x2,y2, "FWHM=" + txt_Kb)

outfig=resDir + "/fit_" + "pL" + str(plen) + "_" + method + str(oflen) + pBstr + "_jitter_bsln.png"
fig.savefig(outfig)
fig.tight_layout()

## 5. Recalibrate energies:
      - get new gain scale with fitted line centres
      - recalibrate energies
      - fit again to get FWHM more precisely

In [ ]:
# reverse au to energy transformation of lines centres using gain scale
#lines_centres_Kas 
# Ecal = coefs[0] + coefs[1] * PHau + coefs[2] * PHau^2 + ....
# to get roots:    0 = coefs[0]-Ecal + coefs[1] * PHau + coefs[2] * PHau^2 + ....
PHau = np.zeros(nlines)
newcoefs = np.zeros(len(coefs))
np.copyto(newcoefs, coefs)
print(coefs)
for i in range(nlines):  
    line = lines_centres_vv[i]
    #print("line=", line)
    coef0 = coefs[0] - line
    newcoefs[0] = coef0
    calpoly = P(newcoefs)
    roots = calpoly.roots()
    #print("Roots=", roots)
    PHau[i] = roots[np.abs(line-roots).argmin()]
    #PHau[i] = max(calpoly.roots())
    #if abs(line - PHau[i]) > 1000:
    #    PHau[i] = min(calpoly.roots())
    #if abs(line - PHau[i]) > 5000:
    #    raise RuntimeError("PH root in a.u. is too different from line Energy (eV)")
    
    #print(calpoly.roots())
    #print("PH for ", line, "=", PHau[i])

In [ ]:
# create NEW gain scale Ka2, Ka1, Kb
plt.close()
ph_lines = np.sort(PHau)
print(ph_lines)
true_lines = np.sort(np.concatenate((MnKas.energies_eV, MnKb.energies_eV)))
print(true_lines)
coefs_final = gainScalePolyFit(xData=ph_lines, yData=true_lines, deg=deg, ylab="MnK Lines energies (eV)")
print("gain scale coefs=", coefs_final)


In [ ]:
# FINAL recalibration of energies
enerKas_final = np.zeros(len(dataKas_HR_jitter_bsln))
enerKb_final = np.zeros(len(dataKb_HR_jitter_bsln))

for i in range(len(coefs_final)):
    enerKas_final += coefs_final[i] * (1e3*dataKas_HR_jitter_bsln)**(i)
    #enerKas_final = inter + slope * 1e3*dataKas_HR_jitter_bsln # eV
enerKas_final = enerKas_final[(enerKas_final > min_Ka) & (enerKas_final < max_Ka)]

for i in range(len(coefs_final)):
    enerKb_final += coefs_final[i] * (1e3*dataKb_HR_jitter_bsln)**(i)
    #enerKb_final = inter + slope * 1e3*dataKb_HR_jitter_bsln # eV
enerKb_final = enerKb_final[(enerKb_final>min_Kb) & (enerKb_final<max_Kb)]


In [ ]:
#%%script false --no-raise-error

# Calculate meadian 'fit' for range of acceptable number of bins in Kas range
#                                                               ===========
(ibmin, ibmax) = (60, 200)
fwhm_G = list()
err_fwhm_G = list()
nbins = list()
for nbinsKas in np.arange(ibmin, ibmax, 10):
    fw, efw, vv_fit = fitVoigt2hist(data=enerKas_final, lines=MnKas, nbins=int(nbinsKas), ax0=ax1)    
    plt.clf()
    if fw < 0 or efw == 0:
        continue
    fwhm_G.append(fw)
    err_fwhm_G.append(efw)
    nbins.append(nbinsKas)
print("FWHM_Gs=", np.array2string(np.asarray(fwhm_G), formatter={'float_kind':lambda x: "%.3f" % x}))
median_fwhm_G = np.median(fwhm_G)
median_err_fwhm_G = np.sum(abs(np.asarray(fwhm_G)-median_fwhm_G))/len(fwhm_G)
median_index = min(range(len(fwhm_G)), key=lambda i: abs(fwhm_G[i]-median_fwhm_G))
nbinsKas = nbins[median_index]
print("median_index=", median_index)
print("FWHM_G[median_index]=",fwhm_G[median_index])
print("ERR_FWHM_G[median_index]=",err_fwhm_G[median_index])
print("nbinsKas[median_index]=", nbins[median_index])

print("Using ", len(fwhm_G), "different number of bins")
txt_Kas = '{:0.2f}'.format(median_fwhm_G) + "+/-" + '{:0.2f}'.format(max(median_err_fwhm_G, err_fwhm_G[median_index])) + "eV"
print("Median FWHM_G(Kas)=", txt_Kas)


In [ ]:
#%%script false --no-raise-error
plt.close()
# Plot representative (median) fitting 
#                    =====================
fig = plt.figure(figsize=(12, 6.5))
ax1 = fig.add_subplot(1,2,1)

fwhm_G, err_fwhm_G, vvmod = fitVoigt2hist(data=enerKas_final, lines=MnKas, nbins=int(nbinsKas), ax0=ax1)
print(fwhm_G, err_fwhm_G)
x1 = min(enerKas_final)
y1 = 20
ax1.text(x1,y1, "FWHM=" + txt_Kas + "\n Total counts=" + str(len(enerKas_final)))
ax1.axvline(MnKas.energies_eV[0], ls="--", color="gray")
ax1.set_title("Representative 'median' fitting")

outfig=resDir + "/fit_" + "pL" + str(plen) + "_" + method + str(oflen) + pBstr + "_final.png"
fig.tight_layout()
fig.savefig(outfig)

In [ ]:
#%%script false --no-raise-error
# Plot representative fitting for a fixed size bin (only for Kas)
#                                   ================
plt.close()
fig = plt.figure(figsize=(13,6.5))
ax1 = fig.add_subplot(1,2,1)

nbinsKas = 80
fwhm_G, err_fwhm_G, vvmod = fitVoigt2hist(data=enerKas_final, lines=MnKas, nbins=int(nbinsKas), ax0=ax1)
print(fwhm_G, err_fwhm_G)

txt_Kas = '{:0.2f}'.format(fwhm_G) + "+/-" + '{:0.2f}'.format(err_fwhm_G) + "eV"
x1 = min(enerKas_HR_jitter_bsln)
y1 = 10
ax1.text(x1,y1, "FWHM=" + txt_Kas + "\n Total counts=" + str(len(enerKas_final)))
ax1.axvline(MnKas.energies_eV[0], ls="--", color="gray")
ax1.set_title("Plot representative -fixed bin- fitting")


outfig=resDir + "/fitKas_" + "pL" + str(plen) + "_" + method + str(oflen) + pBstr + "_final_300bins.png"
fig.tight_layout()
#fig.savefig(outfig)

### 6. Plot ALL calibrated energies

In [ ]:
# Events energy estimation
plt.close()
ener_final = np.zeros(len(data_HR))

for i in range(len(coefs_final)):
    ener_final += coefs_final[i] * (1e3*data_HR.SIGNAL)**(i)

# Have a look to histogram of ALL reconstructed data
fig = plt.figure(figsize=(8,4))
ax1 = fig.add_subplot(1, 1, 1)
npulses = len(data_HR.SIGNAL)
ax1.set_xlabel("Calibrated energy")
ax1.set_ylabel("# events")
ax1.set_title(("Distribution of ALL events (" + str(npulses) + ")"))
ax1.hist(ener_final, bins=4000, density=False, label="Histogram", alpha=0.5)
plotMainXlines(ax1, alp_l=0.3, alp_t=0.6)
fig.tight_layout()

### 7. Plot FWHM for different recontruction methods

In [ ]:
#%%script false --no-raise-error
# plot variation of FWHM for different record lengths and for OPTFILT and preBuffer

rlen      = (4096)
# 300 bins fit
#optf =     (7.12, 3.03, 2.80, 2.74, 2.85, 2.85)
#optf_err = (0.14, 0.07, 0.08, 0.06, 0.07, 0.07)
#pB =       (9.19, 2.86, 2.64, 2.57, 2.71, 2.78)
#pB_err =   (0.18, 0.07, 0.07, 0.07, 0.07, 0.08)

optf =     (3.30)
optf_err = (0.07)
pB =       (3.15)
pB_err =   (0.07)

fig = plt.figure(figsize=(12,9))
ax1 = fig.add_subplot(1, 1, 1)
ax1.errorbar(rlen, optf, yerr=optf_err, label="ADC OPTFILT", marker='o',capsize=3, ls="-", alpha=0.5)
ax1.errorbar(rlen, pB, yerr=pB_err, label="ADC preBuffer75 OPTFILT", marker='o',capsize=2, ls="-")
ax1.tick_params(axis='both', which='major', labelsize=14)
ax1.set_ylim(2.5,10)
ax1.set_xlabel("record length (samples)",fontsize='x-large')
ax1.set_ylabel("FWHM (eV)", fontsize='x-large')
ax1.legend()
#ax1.grid()

axins = inset_axes(ax1, width="70%", height=1.9, bbox_transform=ax1.transAxes, bbox_to_anchor=(0.4, 0.3, 0.5, 0.9), loc=4)
axins.errorbar(rlen, optf, yerr=optf_err, label="ADC OPTFILT", marker='o',capsize=3, ls="-", alpha=0.5)
axins.errorbar(rlen, pB, yerr=pB_err, label="ADC preBuffer75 OPTFILT", marker='o',capsize=2, ls="-")
axins.set_ylim(2.5,4)
axins.grid()
axins.set_xlabel("record length (samples)",fontsize='large')
axins.set_ylabel("FWHM (eV)", fontsize='large')

outfig=resDir + "/fitKas_comparative_final_300bins.png"
fig.savefig(outfig)


In [ ]:
# set up plot
fig, ax = plt.subplots(figsize=(6, 4))
ax.set_ylim([-4, 4])
ax.grid(True)
 
# generate x values
x = np.linspace(0, 2 * np.pi, 100)
 
 
def my_sine(x, w, amp, phi):
    """
    Return a sine for x with angular frequeny w and amplitude amp.
    """
    return amp*np.sin(w * (x-phi))
 
 
@widgets.interact(w=(0, 10, 1), amp=(0, 4, .1), phi=(0, 2*np.pi+0.01, 0.01))
def update(w = 1.0, amp=1, phi=0):
    """Remove old lines from plot and plot new one"""
    [l.remove() for l in ax.lines]
    ax.plot(x, my_sine(x, w, amp, phi), color='C0')